In [3]:
from dae import DAE
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from dataprocessor import dataProcessor
from math import exp, pow

# import csv2utf8
# already created utf8 files

proc = dataProcessor()
data_0, col_list = proc.preProcessing(data_path='train_utf.csv', isTrain = True)
origin_data = proc.train_ref

rand_pick = np.random.permutation(np.arange(data_0.shape[0]))
missing_index = proc.missing_index.loc[rand_pick[:16]]
data_0_test = data_0.loc[rand_pick[:16]]
test_origin = origin_data.loc[rand_pick[:16]]
data_0.drop(rand_pick[:16],inplace=True)

imputer = Midas()
imputer.modeling(origin_data = data_0, num_levels = col_list)

imputer.training(epochs=50)


imputer.restoration(data_0_test)


test_pred = imputer.y_batch



test_pred_row = proc.postProcessing(test_pred)

test_origin_num = test_origin.drop(proc.categorical, axis=1)
test_origin_cat = test_origin[proc.categorical]

test_pred_row_num = test_pred_row.drop(proc.categorical, axis=1)
test_pred_row_cat = test_pred_row[proc.categorical]

missing_index_num = missing_index.drop(proc.categorical, axis=1)
missing_index_cat = missing_index[proc.categorical]




test_origin_cat.reset_index(drop=True, inplace=True)
test_origin_num.reset_index(drop=True, inplace=True)
missing_index_cat.reset_index(drop=True, inplace=True)
num_diff = test_origin_num[missing_index_num==1]-test_pred_row_num[missing_index_num==1]
for column in test_origin_num.columns:
    for index in range(test_origin_num.shape[0]):
        num_diff.loc[index, column] = exp(-pow(num_diff.loc[index, column],2))

numerical_score = num_diff.sum().sum()        

cat_diff = pd.DataFrame(np.zeros_like(test_origin_cat), columns=test_origin_cat.columns)
for column in test_origin_cat.columns:
    for index in range(test_origin_cat.shape[0]):
        if test_origin_cat.loc[index, column]==test_pred_row_cat.loc[index, column] and missing_index_cat.loc[index, column]==1:
            cat_diff.loc[index,column]=1
            
categorical_score = cat_diff.sum().sum()



print('Numeric score: ',numerical_score)
print('Categoric score: ',categorical_score)

generate missing parts
convert to onehot data
0 to 1 scaling


/anaconda/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch: 0 , loss: 27.8332003972 , mse: 1.43811408898 , ce: 21.2151079034 , l2p: 5.18212069698
Epoch: 1 , loss: 24.0773425174 , mse: 1.57726010366 , ce: 17.4632106666 , l2p: 5.01100936008
Epoch: 2 , loss: 22.8625426748 , mse: 1.3483445878 , ce: 16.6021670001 , l2p: 4.89143108483
Epoch: 3 , loss: 21.8022653589 , mse: 1.25082891461 , ce: 15.7475421417 , l2p: 4.78734638463
Epoch: 4 , loss: 20.9853932175 , mse: 1.20759511024 , ce: 15.0253647416 , l2p: 4.68969476882
Epoch: 5 , loss: 20.2677739302 , mse: 1.13708900177 , ce: 14.5119202101 , l2p: 4.59478995788
Epoch: 6 , loss: 19.7090053175 , mse: 1.04436852225 , ce: 14.1353897977 , l2p: 4.50405824125
Epoch: 7 , loss: 19.279685677 , mse: 0.952131225536 , ce: 13.8812435236 , l2p: 4.41692720106
Epoch: 8 , loss: 18.8000340773 , mse: 0.885211551908 , ce: 13.6103536999 , l2p: 4.334244661
Epoch: 9 , loss: 18.4152682606 , mse: 0.813966144569 , ce: 13.3774812162 , l2p: 4.25423065502
Epoch: 10 , loss: 18.0558161712 , mse: 0.736932336835 , ce: 13.14319341

In [4]:
print(missing_index_cat)
print(test_origin_num[missing_index_num==1])
print(test_pred_row_num[missing_index_num==1])

   주야 요일 발생지시도 발생지시군구 사고유형_대분류 사고유형_중분류 법규위반 도로형태_대분류 도로형태 당사자종별_1당_대분류  \
0   0  0     0      0        0        1    0        0    0            0   
1   1  0     0      0        0        0    0        1    0            1   
2   0  0     0      0        0        1    0        0    1            0   
3   0  0     0      0        0        0    0        0    0            0   
4   0  0     0      0        0        0    1        0    0            0   
5   1  1     0      0        1        0    0        0    0            0   
6   0  0     0      0        0        0    0        1    1            1   
7   0  1     1      0        0        0    0        0    0            0   
8   0  0     0      0        0        1    0        0    0            0   
9   1  0     0      0        0        1    0        0    0            1   
10  0  0     1      0        0        1    0        1    0            0   
11  0  0     0      0        1        0    0        0    1            0   
12  0  0     1      0    